In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_filter_map_plot

from IPython.display import display, Markdown, Latex, HTML
import json

In [ ]:
def set_id_date(input_id):
    global analysis_date
    global itp_id
    itp_id = input_id
    if itp_id in [61, 301]:
        analysis_date = dt.date(2022, 6, 8) ##wednesday, new tables for Napa operators
    elif itp_id == 273:
        analysis_date = dt.date(2022, 2, 8) ## override for SacRT
    elif itp_id == 208:
        analysis_date = dt.date(2022, 7, 20) ## override for MST
    elif itp_id in [282, 293]:
        analysis_date = dt.date(2022, 9, 7) ## override for SFMTA
    elif itp_id in [112]:
        analysis_date = dt.date(2022, 9, 20) ## override for foothill (new data!)
    else:
        analysis_date = dt.date(2022, 6, 1) #rebuild site
    return

# Corridors to support bus speeds presentation

* also potential for 100 buses inclusion

## Metro - Santa Monica Bl

In [ ]:
set_id_date(182)

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
rt_day.add_corridor(gpd.read_file('./corridors_100buses/santamonica_hollywood.geojson'))

In [ ]:
%%capture
rt_day.set_filter(start_time='15:00', end_time='19:00', route_names=['4'])

In [ ]:
rt_day.corridor_metrics()

In [ ]:
## avoid papermill errors when map fails to render
m = None

In [ ]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds', no_title=True, corridor=True)

In [ ]:
m

## AC Transit - San Pablo Ave

In [ ]:
set_id_date(4)

In [ ]:
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
rt_day.set_filter(start_time='15:00', end_time='19:00', route_names=['72', '72R'])

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=True)

In [ ]:
m

In [ ]:
corridor = gpd.read_file('./corridors_100buses/sanpablo.geojson')

In [ ]:
rt_day.add_corridor(corridor)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
1922 / 60

In [ ]:
# rt_day._filter(rt_day.rt_trips)

## Sacramento RT - Watt Ave / I80 / 80 biz

In [ ]:
importlib.reload(rt_filter_map_plot)

In [ ]:
set_id_date(273)

In [ ]:
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
rt_day.add_corridor(gpd.read_file('./corridors_100buses/watti80.geojson'), hotspot = True)

In [ ]:
rt_day.set_filter(start_time='15:00', end_time='19:00', route_names=['26', '84', '93'])

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=True, corridor = True)

In [ ]:
m

In [ ]:
rt_day.corridor_metrics()

## SBMTD State/Hollister

In [ ]:
set_id_date(293)

### State Hotspot

In [ ]:
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
rt_day.add_corridor(gpd.read_file('./corridors_100buses/state.geojson'), hotspot = True)

In [ ]:
rt_day.rt_trips.route_short_name.unique()

In [ ]:
rt_day.set_filter(start_time='15:00', end_time='19:00', route_names=['7', '12X', '24X', '15X', '6', '11'])

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=True, corridor = True)

In [ ]:
m

In [ ]:
# rt_day._filter(rt_day.rt_trips)

In [ ]:
# avoid counting freeway route in intersection delay
rt_day.stop_delay_view = rt_day.stop_delay_view >> filter(_.route_id != '7')

In [ ]:
rt_day.corridor_metrics()

### Hollister Hotspot

In [ ]:
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
rt_day.add_corridor(gpd.read_file('./corridors_100buses/hollister.geojson'), hotspot = True)

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=True, corridor = True)

In [ ]:
m

In [ ]:
shp53 = rt_day.stop_delay_view >> filter(_.shape_id == 'shp-12X-53', _.corridor, _.stop_sequence > 3.9)
shp04 = rt_day.stop_delay_view >> filter(_.shape_id == 'shp-12X-04', _.corridor, _.stop_sequence < 9.1)

In [ ]:
# rt_day.stop_delay_view >> filter(_.route_short_name == '12X', _.corridor) >> distinct(_.stop_sequence, _keep_all=True) >> arrange(_.shape_id, _.stop_sequence)

In [ ]:
rt_day.stop_delay_view = rt_day.stop_delay_view >> filter(-_.shape_id.isin(['shp-12X-53', 'shp-12X-04']))

In [ ]:
rt_day.stop_delay_view = pd.concat([rt_day.stop_delay_view, shp53, shp04])

In [ ]:
rt_day.corridor_metrics()